<a href="https://colab.research.google.com/github/francescomazzolin/stochastic-finance-project/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Stochastic Finance Project

Giada Martini

Francesco Mazzolin

Francesco Salvagnin

Nicolas Stecca


## Importing the packages


In [1]:

#!pip install eikon

import eikon as ek
import pandas as pd
import numpy as np
from scipy.stats import norm



## Retrieving the data

In [2]:
# Setting up the API key
ek.set_app_key('6e2b6a629eb84c0b859dc5faf22d48f94d85de97')





c:\Users\paolo\anaconda3\Lib\site-packages\eikon\data_grid.py:255: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors='ignore')
c:\Users\paolo\anaconda3\Lib\site-packages\eikon\data_grid.py:255: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors='ignore')


In [ ]:

def get_financial_data(rics):
    rics = rics
    #Data fields we are interested in: 
      #Total value of equity
      #Total value of debt
    fields = ['TR.F.ComEqTot', 'TR.F.DebtTot']
    results = []

    for ric in rics:
        data, err = ek.get_data(instruments=[ric], fields=fields)
        if err is None:
            data['RIC'] = ric
            results.append(data)
        else:
            print(f"Error retrieving data for {ric}: {err}")

    #Combine results into a single DataFrame
    df = pd.concat(results, ignore_index=True)
    df.drop(columns=['RIC'])

    #Add Total asset value column
    df['Total_value'] = df['Common Equity - Total'] + df['Debt - Total']

    return df


#### Choosing the stocks

In [ ]:
rics_list = ['AAPL.O','FB.O']



#### Retrieving the data

In [ ]:
data = get_financial_data(rics_list)

In [4]:
print(data.head())

  Instrument  Common Equity - Total  Debt - Total     RIC   Total_value
0     AAPL.O            56950000000  107525000000  AAPL.O  164475000000
1       FB.O           153168000000   19075000000    FB.O  172243000000
